In [211]:
import pandas as pd
import os

data_dir = "../RxNorm_full_01032023/rrf/"

### RXNCONSO.RFF

In [212]:
# read dir
conso_dir = os.path.join(data_dir, 'RXNCONSO.RRF')
conso_df = pd.read_csv(conso_dir, delimiter='|', header=None)

print(f'Read {len(conso_df):,} lines from {conso_dir}.')

# set columns
col_list = list(conso_df.columns)
col_list[0] = 'RXCUI'
col_list[7] = 'RXAUI'
col_list[14] = 'TERM'
conso_df.columns = col_list

# filter
conso_df = conso_df[['RXCUI', 'RXAUI', 'TERM']]

# drop NaN
conso_df.dropna(inplace=True)
print(f'Read {len(conso_df):,} lines without NaN from {conso_dir}.')



Read 1,125,456 lines from ../RxNorm_full_01032023/rrf/RXNCONSO.RRF.
Read 1,125,456 lines without NaN from ../RxNorm_full_01032023/rrf/RXNCONSO.RRF.


### RXNREL.RFF

In [213]:
# read dir
rel_dir = os.path.join(data_dir, 'RXNREL.RRF')
rel_df = pd.read_csv(rel_dir, delimiter='|', header=None)

print(f'Read {len(rel_df):,} lines from {rel_dir}.')

# set columns
col_list = list(rel_df.columns)
col_list[0] = 'UI1_CUI'
col_list[1] = 'UI1_AUI'
col_list[2] = 'UI1_TYPE'
col_list[4] = 'UI2_CUI'
col_list[5] = 'UI2_AUI'
col_list[6] = 'UI2_TYPE'
col_list[7] = 'REL'
rel_df.columns = col_list

# filter
rel_df = rel_df[['UI1_AUI', 'UI1_CUI', 'UI1_TYPE', 'UI2_AUI', 'UI2_CUI', 'UI2_TYPE', 'REL']]

# assert all relations are between similar types
assert (rel_df['UI1_TYPE'] == rel_df['UI2_TYPE']).all()

# only keep CUI relations
rel_df = rel_df[rel_df['UI1_TYPE'] == 'CUI']
print(f'Read {len(rel_df):,} CUI relations.')

# rename again
rel_df = rel_df[['UI1_CUI', 'UI2_CUI', 'REL']]
rel_df.columns = ['CUI1', 'CUI2', 'REL']

# drop NaN
rel_df.dropna(inplace=True)
print(f'Read {len(rel_df):,} CUI relations without NaN.')

# cast column
rel_df = rel_df.astype({
    'CUI1': 'int32',
    'CUI2': 'int32',
})


/var/folders/xp/33m08yl56t1f5z8g9s2h_1j80000gq/T/ipykernel_1632/262334205.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  rel_df = pd.read_csv(rel_dir, delimiter='|', header=None)


Read 7,373,670 lines from ../RxNorm_full_01032023/rrf/RXNREL.RRF.
Read 1,471,262 CUI relations.
Read 1,471,262 CUI relations without NaN.


### Some data exploration

In [214]:
# number of CUI
print(f'Found {len(conso_df["RXCUI"].unique()):,} unique RXCUIs.')
# number of AUI
print(f'Found {len(conso_df["RXAUI"].unique()):,} unique RXAUIs.')
# number of unique terms
print(f'Found {len(conso_df["TERM"].unique()):,} unique TERMs.')

# CUIs participating in relations
unique_values_in_relations = pd.concat([rel_df['CUI1'], rel_df['CUI2']]).unique()

print(f'Found {len(unique_values_in_relations):,} unique RXCUIs participating in relations.')


Found 380,965 unique RXCUIs.
Found 1,125,456 unique RXAUIs.
Found 938,113 unique TERMs.
Found 195,120 unique RXCUIs participating in relations.


In [ ]:
conso_df

In [ ]:
# check that each term is only related to one concept?
grouped_df = conso_df.copy()
# get a list of all IDS per term
grouped_df.drop(columns=['RXAUI'], inplace=True)
grouped_df['RXCUI'] = grouped_df['RXCUI'].apply(lambda x: [x])
grouped_df = grouped_df.groupby('TERM').sum()
grouped_df['UNIQUE_RXCUI'] = grouped_df['RXCUI'].apply(lambda x: len(set(x)))

# check where the number of concepts is not 1
grouped_df_duplicates = grouped_df[grouped_df['UNIQUE_RXCUI'] > 1]

In [ ]:
grouped_df_duplicates

In [ ]:
# unique CUI relations
print(f'Found {len(rel_df["REL"].unique()):,} unique CUI relations.')
print(f'Unique CUI relations: {rel_df["REL"].unique()}')

In [ ]:
# NOTE: procedure can be optimized
def lookup(term, conso_df):
    # find the concept related to the therm
    unique_concept_matches = conso_df[conso_df['TERM'] == term]['RXCUI'].unique()
    concept_to_atoms = {}
    for concept in unique_concept_matches:
        # find all atoms under this concept
        atoms = conso_df[conso_df['RXCUI'] == concept]
        concept_to_atoms.update({
            concept: atoms
        })
    return concept_to_atoms

In [ ]:
concepts = lookup('Acetaminophen', conso_df)
print(concepts.keys())
for concept in concepts.keys():
    print(concept,':')
    print(concepts[concept].head(5))

In [ ]:
concepts = lookup('various', conso_df)
print(concepts.keys())
for concept in concepts.keys():
    print(concept,':')
    print(concepts[concept].head(5))

In [ ]:
concepts = lookup('watermelon', conso_df)
print(concepts.keys())
for concept in concepts.keys():
    print(concept,':')
    print(concepts[concept].head(5))

In [208]:
def get_relation(cui1, cui2, rel_df):
    forward_relation = rel_df[(rel_df['CUI1'] == cui1) & (rel_df['CUI2'] == cui2)]
    backward_relation = rel_df[(rel_df['CUI2'] == cui1) & (rel_df['CUI1'] == cui2)]
    assert len(forward_relation) == len(backward_relation)
    return forward_relation, backward_relation

def get_all_relations(cui1, rel_df):
    forward_relations = rel_df[(rel_df['CUI1'] == cui1)]
    backward_relations = rel_df[(rel_df['CUI2'] == cui1)]
    assert len(forward_relations) == len(backward_relations)
    return forward_relations, backward_relations

def get_all_concepts(term, conso_df):
    unique_concept_matches = conso_df[conso_df['TERM'] == term]['RXCUI'].unique()
    return list(unique_concept_matches)

def get_all_terms(cui1, conso_df):
    atoms = conso_df[conso_df['RXCUI'] == cui1]['TERM'].unique()
    return list(atoms)

In [209]:
def get_all_related_concepts(cui, rel_df):
    forward_relations, _ = get_all_relations(cui, rel_df)
    related_concepts = forward_relations['CUI2'].unique()
    return list(related_concepts)

def get_all_related_atoms(term, conso_df, rel_df):
    parent_concepts = get_all_concepts(term, conso_df)
    rel_concepts = []
    for concept in parent_concepts:
        rel_concepts.extend(get_all_related_concepts(concept, rel_df))
    rel_concepts.extend(parent_concepts)
    rel_atoms = []
    for concept in rel_concepts:
        rel_atoms.extend(get_all_terms(concept, conso_df))
    return list(set(rel_atoms))

# def get_all_related_atoms()

In [210]:
get_all_related_atoms('paracetamol', conso_df, rel_df)

['APAP/brompheniramine/phenylpropanolamine',
 'Calpol Six Plus',
 'acetaminophen / diphenhydramine / pseudoephedrine Chewable Product',
 'acetaminophen 6.67 MG/ML',
 'ACETAMINOPHEN/DEXTROMETHORPHAN/PHENYLEPHRINE/PYRILAMINE',
 'Calpol',
 'Oxycodone- and paracetamol-containing product',
 'acetaminophen / meperidine',
 'acetaminophen / dextromethorphan / phenylephrine / triprolidine Oral Capsule',
 'Anacin Advanced Headache Formula',
 'acetaminophen / dextromethorphan / guaifenesin / pseudoephedrine Oral Solution',
 'acetaminophen / dextromethorphan / doxylamine / pseudoephedrine',
 'Panadol Extra',
 'ACETAMINOPHEN/CHLORPHENIRAMINE/CODEINE/PHENYLEPHRINE',
 'acetaminophen / diphenhydramine Oral Powder Product',
 'acetaminophen / diphenhydrAMINE Oral Liquid Product',
 'Tylenol Cold Relief Nighttime Caplet',
 'acetaminophen / pamabrom Extended Release Oral Tablet',
 'acetaminophen 26 MG/ML',
 'acetaminophen / diphenhydramine Pill',
 'Margesic-H',
 'Flextra Plus',
 'ACETAMINOPHEN/BUTALBITAL',

### Try to solve our problems with Join
Try to build a large LUT.

One row: term | parent cui | related cui | relation | related_term

How to build:
1. merge term | aui | cui with cui1 | rel | cui2
2. merge result with term | aui | cui

Add identity relation to the relation table to not have to build an exception here. If it proves to be too heavy, remove inverse relations.

In [268]:
# add reflexive relation
unique_cuis = conso_df["RXCUI"].unique()
rel_reflexive_df = pd.DataFrame(data= {"CUI1":unique_cuis,"CUI2":unique_cuis})
rel_reflexive_df['REL'] = 'is_same_concept'

# only keep unique (RXCUI, TERM) combinations
conso_df_unique = conso_df.drop(columns=['RXAUI']).groupby(['RXCUI', 'TERM']).first().reset_index()

In [272]:
rel_df_left = rel_df.copy()
# add reflexive relation
rel_df_left = pd.concat([rel_df_left, rel_reflexive_df])
# prepare for merge
rel_df_left = rel_df_left.rename(columns={
    'CUI1':'RXCUI'
})

conso_df_right = conso_df_unique.copy()
# prepare for merge
conso_df_right = conso_df_unique.rename(columns={
    'RXCUI':'RXCUI2',
    'TERM':'TERM2'
})

In [273]:
merged_df = conso_df_unique.reset_index().merge(rel_df_left, on='RXCUI', how='outer').set_index('index')
merged_df = merged_df.rename(columns={
  'RXCUI': 'RXCUI1',  
  'TERM': 'TERM1',  
  'CUI2': 'RXCUI2'
})
merged_df = merged_df.reset_index().merge(conso_df_right, on='RXCUI2', how='outer').set_index('index')

In [274]:
merged_df

,RXCUI1,TERM1,RXCUI2,REL,TERM2
index,,,,,
0,3,"1,4-alpha-Glucan branching enzyme",3,is_same_concept,"1,4-alpha-Glucan branching enzyme"
0,3,"1,4-alpha-Glucan branching enzyme",3,is_same_concept,"1,4-alpha-Glucan branching enzyme (substance)"
0,3,"1,4-alpha-Glucan branching enzyme",3,is_same_concept,"Amylo-(1,4,6)-transglycosylase"
0,3,"1,4-alpha-Glucan branching enzyme",3,is_same_concept,Branching enzyme
1,3,"1,4-alpha-Glucan branching enzyme (substance)",3,is_same_concept,"1,4-alpha-Glucan branching enzyme"
...,...,...,...,...,...
941476,2624752,"Ebola Zaire Vaccine, Live Intramuscular Suspen...",2624749,constitutes,Zaire ebolavirus (strain Kikwit-95) envelope g...
941477,2624752,"Ervebo 72,000,000 UNT per 1 ML Injection",2624749,constitutes,Zaire ebolavirus (strain Kikwit-95) envelope g...
941478,2624752,ZAIRE EBOLAVIRUS (STRAIN KIKWIT-95) ENVELOPE G...,2624749,constitutes,Zaire ebolavirus (strain Kikwit-95) envelope g...


In [275]:


def get_all_relations(term, merged_df):
    return merged_df[merged_df['TERM1'] == term]

def get_links(term1, term2, merged_df):
    return merged_df[(merged_df['TERM1'] == term1) & (merged_df['TERM2'] == term2)]

In [276]:
get_all_relations('ibuprofen', merged_df)

,RXCUI1,TERM1,RXCUI2,REL,TERM2
index,,,,,
13660,5640,ibuprofen,372450,has_ingredient,acetaminophen / ibuprofen Oral Capsule
13660,5640,ibuprofen,372451,has_ingredient,acetaminophen / ibuprofen Oral Suspension
13660,5640,ibuprofen,372452,has_ingredient,acetaminophen / ibuprofen Oral Tablet
13660,5640,ibuprofen,818102,has_part,ACETAMINOPHEN/IBUPROFEN
13660,5640,ibuprofen,818102,has_part,Acetaminophen- and ibuprofen-containing product
...,...,...,...,...,...
13660,5640,ibuprofen,5640,is_same_concept,alpha-Methyl-4-(2-methylpropyl)benzeneacetic Acid
13660,5640,ibuprofen,5640,is_same_concept,ibuprofen
13660,5640,ibuprofen,5640,is_same_concept,p-Isobutylhydratropic acid


In [277]:
get_all_relations('ibuprofen', merged_df)['TERM2'].unique()

array(['acetaminophen / ibuprofen Oral Capsule',
       'acetaminophen / ibuprofen Oral Suspension',
       'acetaminophen / ibuprofen Oral Tablet', 'ACETAMINOPHEN/IBUPROFEN',
       'Acetaminophen- and ibuprofen-containing product',
       'Ibuprofen- and paracetamol-containing product',
       'Product containing ibuprofen and paracetamol (medicinal product)',
       'acetaminophen / ibuprofen', 'acetaminophen-ibuprofen',
       'acetaminophen / ibuprofen Oral Liquid Product',
       'Acetaminophen- and ibuprofen-containing product in oral dose form',
       'Ibuprofen- and paracetamol-containing product in oral dose form',
       'Product containing ibuprofen and paracetamol in oral dose form (medicinal product form)',
       'acetaminophen / ibuprofen Oral Product',
       'acetaminophen / ibuprofen Pill', 'Motrin PM',
       'caffeine / ergotamine / ibuprofen Oral Tablet',
       'caffeine / ergotamine / ibuprofen',
       'caffeine / ergotamine / ibuprofen Oral Product',
       '

In [278]:
get_links('ibuprofen', 'ACETAMINOPHEN/IBUPROFEN', merged_df)

,RXCUI1,TERM1,RXCUI2,REL,TERM2
index,,,,,
13660,5640,ibuprofen,818102,has_part,ACETAMINOPHEN/IBUPROFEN


In [279]:
get_links('ibuprofen', 'carisoprodol / ibuprofen Pill', merged_df)

,RXCUI1,TERM1,RXCUI2,REL,TERM2
index,,,,,
13660,5640,ibuprofen,1151406,has_ingredient,carisoprodol / ibuprofen Pill


In [281]:
get_all_relations('watermelon', merged_df)

,RXCUI1,TERM1,RXCUI2,REL,TERM2
index,,,,,
168615,260014,watermelon,260014,is_same_concept,Watermelon
168615,260014,watermelon,260014,is_same_concept,watermelon
168615,260014,watermelon,260014,is_same_concept,watermelon preparation
441569,901258,watermelon,901263,has_ingredient,watermelon allergenic extract Injectable Solution
441569,901258,watermelon,901259,has_ingredient,watermelon allergenic extract 50 MG/ML
441569,901258,watermelon,901262,has_ingredient,watermelon allergenic extract 100 MG/ML
441569,901258,watermelon,1164599,has_ingredient,watermelon allergenic extract Injectable Product
441569,901258,watermelon,901258,is_same_concept,WATERMELON
441569,901258,watermelon,901258,is_same_concept,Watermelon


In [282]:
get_all_relations('PHENYLEPHRINE', merged_df)

,RXCUI1,TERM1,RXCUI2,REL,TERM2
index,,,,,
20255,8163,PHENYLEPHRINE,214186,has_part,ACETAMINOPHEN/PHENYLEPHRINE
20255,8163,PHENYLEPHRINE,214186,has_part,Acetaminophen- and phenylephrine-containing pr...
20255,8163,PHENYLEPHRINE,214186,has_part,Paracetamol- and phenylephrine-containing product
20255,8163,PHENYLEPHRINE,214186,has_part,Product containing paracetamol and phenylephri...
20255,8163,PHENYLEPHRINE,214186,has_part,acetaminophen / phenylephrine
...,...,...,...,...,...
20255,8163,PHENYLEPHRINE,8163,is_same_concept,Phenylephrinum
20255,8163,PHENYLEPHRINE,8163,is_same_concept,Product containing phenylephrine (medicinal pr...
20255,8163,PHENYLEPHRINE,8163,is_same_concept,R(-)-Phenylephrine


In [283]:
get_all_relations('PHENYLEPHRINE', merged_df)['REL'].unique()

array(['has_part', 'tradename_of', 'has_ingredient', 'form_of',
       'is_same_concept'], dtype=object)